<a href="https://colab.research.google.com/github/ferjorosa/learn-pytorch/blob/main/FastAI%20NLP%20Course/Translation%2C%20attention%20and%20transformers/translation_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Translation with a Sequence to Sequence Network and Attention

In this project we will be teaching a neural network to translate from French to English.

```
[KEY: > input, = target, < output]

> il est en train de peindre un tableau .
= he is painting a picture .
< he is painting a picture .

> pourquoi ne pas essayer ce vin delicieux ?
= why not try that delicious wine ?
< why not try that delicious wine ?

> elle n est pas poete mais romanciere .
= she is not a poet but a novelist .
< she not not a poet but a novelist .

> vous etes trop maigre .
= you re too skinny .
< you re all alone .
```

This is made possible by the simple but powerful idea of the <a href="https://arxiv.org/abs/1409.3215">sequence to sequence network</a>, in which two recurrent neural networks work together to transform one sequence to another. An encoder network condenses an input sequence into a vector, and a decoder network unfolds that vector into a new sequence.

<img src="images/sequence_to_sequence_example.png">

To improve upon this model we’ll use an <a href="https://arxiv.org/abs/1409.0473">attention mechanism</a>, which lets the decoder learn to focus over a specific range of the input sequence.

In [1]:
#Only on Google Colab

URL = "https://raw.githubusercontent.com/ferjorosa/learn-pytorch/main/FastAI%20NLP%20Course/Translation%2C%20attention%20and%20transformers/data_translation"

!wget -r --no-parent --cut-dirs=7 {URL}/eng-fra.txt


!mv raw.githubusercontent.com data_translation


--2022-03-28 17:44:03--  https://raw.githubusercontent.com/ferjorosa/learn-pytorch/main/FastAI%20NLP%20Course/Translation%2C%20attention%20and%20transformers/data_translation/eng-fra.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9541158 (9.1M) [text/plain]
Saving to: ‘raw.githubusercontent.com/eng-fra.txt’

raw.githubuserconte 100%[===================>]   9.10M  --.-KB/s    in 0.07s   

2022-03-28 17:44:04 (138 MB/s) - ‘raw.githubusercontent.com/eng-fra.txt’ saved [9541158/9541158]

FINISHED --2022-03-28 17:44:04--
Total wall clock time: 0.7s
Downloaded: 1 files, 9.1M in 0.07s (138 MB/s)


In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")